In [1]:
# Importing Dependencies
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

from camel import CAMELAgent
from config import assistant_role_name, user_role_name, task
from prompt import task_specifier_sys_msg, task_specifier_msg, get_sys_msgs

In [2]:
# Create a Task Specifing Agent for Brainstorming
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0))
specified_task_msg = task_specify_agent.step(task_specifier_msg)
specified_task = specified_task_msg.content

In [3]:
specified_task

'Write a 300-word blog post highlighting the benefits of using a chatbot on a website, focusing on how it can improve customer engagement, provide instant support, and increase conversion rates. Incorporate real-life examples and statistics to illustrate the impact of chatbots on user experience and business success.'

In [4]:
# Get System and User Inception Messages
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name, specified_task)

# Create an Assistant Agent for Inception
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0))

# Create a User Agent for Inception
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.0))

In [5]:
# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
assistant_msg = HumanMessage(
    content=(
        "Now start to give me instructions one by one. "
        "Only reply with Instruction and Input."
    )
)
assistant_msg

HumanMessage(content='Now start to give me instructions one by one. Only reply with Instruction and Input.')

In [6]:
user_agent.stored_messages

[SystemMessage(content='Never forget you are a Founder and CEO and I am a Content Writer. Never flip roles! You will always instruct me.\nWe share a common interest in collaborating to successfully complete a task.\nI must help you to complete the task.\nHere is the task: Write a 300-word blog post highlighting the benefits of using a chatbot on a website, focusing on how it can improve customer engagement, provide instant support, and increase conversion rates. Incorporate real-life examples and statistics to illustrate the impact of chatbots on user experience and business success.. Never forget our task!\nYou must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:\n\n1. Instruct with a necessary input:\nInstruction: <YOUR_INSTRUCTION>\nInput: <YOUR_INPUT>\n\n2. Instruct without any input:\nInstruction: <YOUR_INSTRUCTION>\nInput: None\n\nThe "Instruction" describes a task or question. The paired "Input" provides further context or in

In [7]:
assistant_agent.stored_messages

[SystemMessage(content='Never forget you are a Content Writer and I am a Founder and CEO. Never flip roles! Never instruct me!\nWe share a common interest in collaborating to successfully complete a task.\nYou must help me to complete the task.\nHere is the task: Write a 300-word blog post highlighting the benefits of using a chatbot on a website, focusing on how it can improve customer engagement, provide instant support, and increase conversion rates. Incorporate real-life examples and statistics to illustrate the impact of chatbots on user experience and business success.. Never forget our task!\nI must instruct you based on your expertise and my needs to complete the task.\n\nI must give you one instruction at a time.\nYou must write a specific solution that appropriately completes the requested instruction.\nYou must decline my instruction honestly if you cannot perform the instruction due to physical, moral, legal reasons or your capability and explain the reasons.\nDo not add an

In [8]:
print(f"Original task prompt:\n{task}\n")
print(f"Specified task prompt:\n{specified_task}\n")

chat_turn_limit, n = 10, 0
assistant_response =[]
while n < chat_turn_limit:
    n += 1
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)
    print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
    assistant_response.append(assistant_msg.content)
    if "<CAMEL_TASK_DONE>" in user_msg.content:
        break

Original task prompt:
Write a short post on the benefits of using a chatbot on a website.

Specified task prompt:
Write a 300-word blog post highlighting the benefits of using a chatbot on a website, focusing on how it can improve customer engagement, provide instant support, and increase conversion rates. Incorporate real-life examples and statistics to illustrate the impact of chatbots on user experience and business success.

AI User (Founder and CEO):

Instruction: Write an engaging introduction about the importance of customer engagement on a website.
Input: None


AI Assistant (Content Writer):

Solution: Customer engagement is crucial for the success of any business website. It is the key to building strong relationships with customers, understanding their needs, and providing them with the support they require. Engaging with customers in real-time can make a significant impact on their overall experience and satisfaction. <NEXT_REQUEST>


AI User (Founder and CEO):

Instruction

In [11]:
output = "".join(assistant_response)

In [12]:
output

"Solution: Customer engagement is crucial for the success of any business website. It is the key to building strong relationships with customers, understanding their needs, and providing them with the support they require. Engaging with customers in real-time can make a significant impact on their overall experience and satisfaction. <NEXT_REQUEST>Solution: Chatbots can improve customer engagement by providing instant responses to inquiries, guiding users through product options, and offering personalized recommendations based on user preferences. For example, a clothing retailer's chatbot can engage customers by helping them find the perfect outfit, offering styling tips, and answering questions about sizing and availability, all in real-time. This level of engagement can significantly enhance the overall customer experience. <NEXT_REQUEST>Solution: Using a chatbot to provide instant support on a website offers the benefit of 24/7 availability, ensuring that customers can receive assi

In [14]:
import re

# Regex pattern to remove "Solution:" and "<NEXT_REQUEST>"
pattern = r"Solution: | <NEXT_REQUEST>"

# Removing the unwanted parts
cleaned_output = re.sub(pattern, "", output)

In [31]:
cleaned_output.split(". ")

['Customer engagement is crucial for the success of any business website',
 'It is the key to building strong relationships with customers, understanding their needs, and providing them with the support they require',
 'Engaging with customers in real-time can make a significant impact on their overall experience and satisfaction.Chatbots can improve customer engagement by providing instant responses to inquiries, guiding users through product options, and offering personalized recommendations based on user preferences',
 "For example, a clothing retailer's chatbot can engage customers by helping them find the perfect outfit, offering styling tips, and answering questions about sizing and availability, all in real-time",
 'This level of engagement can significantly enhance the overall customer experience.Using a chatbot to provide instant support on a website offers the benefit of 24/7 availability, ensuring that customers can receive assistance at any time, leading to increased custom

In [18]:
# Counting the number of words in the cleaned output
len(cleaned_output.split())

253